In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel(r'C:\Users\faiza\Desktop\web-scraping-\final.xlsx')

In [2]:
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora, models
import pyLDAvis.gensim_models
from nltk.corpus import stopwords
import string
from nltk.stem.wordnet import WordNetLemmatizer

import warnings
warnings.simplefilter('ignore')
from itertools import chain

In [3]:
#import nltk
#nltk.download('all')

In [4]:
df.head()

,topic,title,article,date,link
0,BUSINESS,"MAHB faces double whammy of interstate ban, Tu...",KUALA LUMPUR: Affin Hwang Capital expects Mala...,"May 4, 2021 @ 10:40am",https://www.nst.com.my/business/2021/05/687692...
1,BUSINESS,RM opens at 4.09 against USD,KUALA LUMPUR: The ringgit opened easier agains...,"May 3, 2021 @ 10:09am",https://www.nst.com.my/business/2021/05/687398...
2,COLUMNISTS,Biden needs fresh thinking and more than one term,FORGET about the first 100 days. US President ...,"May 3, 2021 @ 12:10am",https://www.nst.com.my/opinion/columnists/2021...
3,SUNDAY VIBES,MONEY THOUGHTS: Ikigai — The uncommon key to c...,HAVE you noticed how life's getting tougher on...,"May 2, 2021 @ 8:30am",https://www.nst.com.my/lifestyle/sunday-vibes/...
4,BUSINESS,"Labuan FSA appoints new chairman, director gen...",KUALA LUMPUR: Labuan Financial Services Author...,"Apr 30, 2021 @ 4:15pm",https://www.nst.com.my/business/2021/04/686731...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   topic    107 non-null    object
 1   title    107 non-null    object
 2   article  107 non-null    object
 3   date     107 non-null    object
 4   link     107 non-null    object
dtypes: object(5)
memory usage: 4.3+ KB


In [6]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(text):
    stop_free = ' '.join([word for word in text.lower().split() if word not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join([lemma.lemmatize(word) for word in punc_free.split()])
    return normalized.split()

In [7]:
df['article_clean']=df['article'].apply(clean)

In [8]:
dictionary = corpora.Dictionary(df['article_clean'])
print(dictionary.num_nnz)

20580


In [9]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in df['article_clean'] ]
print(len(doc_term_matrix))

107


In [10]:
lda = gensim.models.ldamodel.LdaModel

In [11]:
num_topics= 8
%time ldamodel = lda(doc_term_matrix,num_topics=num_topics,id2word=dictionary,passes=10,minimum_probability=0)

Wall time: 1.24 s


In [12]:
ldamodel.print_topics(num_topics=num_topics)

[(0,
  '0.011*"per" + 0.010*"cent" + 0.008*"also" + 0.006*"budget" + 0.005*"billion" + 0.005*"economy" + 0.005*"digital" + 0.005*"2021" + 0.005*"government" + 0.004*"said"'),
 (1,
  '0.018*"said" + 0.010*"malaysia" + 0.009*"u" + 0.008*"per" + 0.008*"currency" + 0.007*"dollar" + 0.007*"cent" + 0.007*"ringgit" + 0.006*"local" + 0.006*"market"'),
 (2,
  '0.013*"said" + 0.011*"per" + 0.011*"cent" + 0.007*"bank" + 0.006*"fund" + 0.006*"police" + 0.006*"market" + 0.006*"year" + 0.005*"malaysia" + 0.005*"bitcoin"'),
 (3,
  '0.012*"ringgit" + 0.010*"dollar" + 0.009*"u" + 0.009*"country" + 0.008*"local" + 0.008*"government" + 0.008*"currency" + 0.007*"covid19" + 0.006*"said" + 0.006*"also"'),
 (4,
  '0.011*"million" + 0.009*"said" + 0.005*"asset" + 0.005*"quarter" + 0.005*"per" + 0.005*"cent" + 0.004*"year" + 0.004*"2020" + 0.004*"people" + 0.004*"malaysia"'),
 (5,
  '0.007*"year" + 0.004*"said" + 0.004*"business" + 0.004*"back" + 0.004*"helicopter" + 0.004*"education" + 0.003*"time" + 0.003*"i

In [13]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary, sort_topics=False, mds='mmds')
pyLDAvis.display(lda_display)

In [14]:
lda_corpus = ldamodel[doc_term_matrix]

In [15]:
[doc for doc in lda_corpus]

[[(0, 0.00055632304),
  (1, 0.9961063),
  (2, 0.00055615045),
  (3, 0.0005562322),
  (4, 0.00055631966),
  (5, 0.00055605586),
  (6, 0.00055620115),
  (7, 0.0005563778)],
 [(0, 0.0008877289),
  (1, 0.08995946),
  (2, 0.0008877603),
  (3, 0.50746083),
  (4, 0.00088743103),
  (5, 0.00088735856),
  (6, 0.00088819314),
  (7, 0.3981412)],
 [(0, 0.00013504077),
  (1, 0.00013503733),
  (2, 0.00013500785),
  (3, 0.00013505087),
  (4, 0.00013501578),
  (5, 0.00013503934),
  (6, 0.00013505387),
  (7, 0.99905473)],
 [(0, 0.00022564233),
  (1, 0.00022564866),
  (2, 0.00022564606),
  (3, 0.00022566304),
  (4, 0.121967725),
  (5, 0.094469),
  (6, 0.00022565827),
  (7, 0.782435)],
 [(0, 0.00088793074),
  (1, 0.0008882804),
  (2, 0.0008878012),
  (3, 0.00088812114),
  (4, 0.00088759913),
  (5, 0.993784),
  (6, 0.0008881675),
  (7, 0.0008881106)],
 [(0, 0.0005957658),
  (1, 0.9958288),
  (2, 0.000595855),
  (3, 0.0005963862),
  (4, 0.00059560244),
  (5, 0.0005955886),
  (6, 0.000595868),
  (7, 0.000596

In [16]:
scores = list(chain(*[[score for topic_id,score in topic] \
                      for topic in [doc for doc in lda_corpus]]))

threshold = sum(scores)/len(scores)
print(threshold)

0.12499999938357934


In [17]:
cluster1 = [j for i,j in zip(lda_corpus,df.index) if i[0][1] > threshold]
cluster2 = [j for i,j in zip(lda_corpus,df.index) if i[1][1] > threshold]
cluster3 = [j for i,j in zip(lda_corpus,df.index) if i[2][1] > threshold]
cluster4 = [j for i,j in zip(lda_corpus,df.index) if i[3][1] > threshold]
cluster5 = [j for i,j in zip(lda_corpus,df.index) if i[4][1] > threshold]

In [18]:
df.iloc[cluster1]

,topic,title,article,date,link,article_clean
11,BUSINESS,RM opens at 4.11 against USD,KUALA LUMPUR: The ringgit opened slightly lowe...,"Apr 23, 2021 @ 10:08am",https://www.nst.com.my/business/2021/04/684656...,"[kuala, lumpur, ringgit, opened, slightly, low..."
27,CRIME & COURTS,Hunt on for Bitcoin mining culprit behind TNB'...,MELAKA: Police here are hunting for a man beli...,"Mar 26, 2021 @ 7:39pm",https://www.nst.com.my/news/crime-courts/2021/...,"[melaka, police, hunting, man, believed, main,..."
33,BUSINESS,"IHH, MAHB shares fall after Turkey's lira take...",KUALA LUMPUR: Khazanah Nasional Bhd's listed f...,"Mar 23, 2021 @ 9:57am",https://www.nst.com.my/business/2021/03/676147...,"[kuala, lumpur, khazanah, nasional, bhds, list..."
53,BOTS,"#TECH: Nyan Cat GIF fetched nearly US$500,000 ...",WHO would have thought that an animated flying...,"Feb 24, 2021 @ 8:49pm",https://www.nst.com.my/lifestyle/bots/2021/02/...,"[would, thought, animated, flying, cat, leavin..."
57,BUSINESS,Business as usual for Parlo in Myanmar,Ayisy Yusof\nKUALA LUMPUR: Parlo Bhd says its ...,"Feb 4, 2021 @ 2:13pm",https://www.nst.com.my/business/2021/02/663031...,"[ayisy, yusof, kuala, lumpur, parlo, bhd, say,..."
78,BUSINESS,"Bitcoin passes US$30,000 for first time","PARIS: Bitcoin, the leading virtual currency, ...","Jan 3, 2021 @ 9:07am",https://www.nst.com.my/business/2021/01/654014...,"[paris, bitcoin, leading, virtual, currency, s..."
87,WORLD,"Bitcoin on record tear towards US$20,000",LONDON: Top virtual currency bitcoin briefly s...,"Dec 1, 2020 @ 9:44pm",https://www.nst.com.my/world/world/2020/12/645...,"[london, top, virtual, currency, bitcoin, brie..."
105,NATION,In good position with largest budget in history,"KUALA LUMPUR: The 2021 Budget, themed 'Resilie...","Nov 15, 2020 @ 8:31am",https://www.nst.com.my/news/nation/2020/11/641...,"[kuala, lumpur, 2021, budget, themed, resilien..."


In [19]:
df.iloc[cluster5]

,topic,title,article,date,link,article_clean
29,NATION,Yemeni civil war more devastating than Covid-1...,KUALA LUMPUR: The raging civil war in Yemen is...,"Mar 25, 2021 @ 5:24pm",https://www.nst.com.my/news/nation/2021/03/676...,"[kuala, lumpur, raging, civil, war, yemen, dev..."
48,BUSINESS,IHH's FY20 net profit falls 47.6pct to RM288.8...,KUALA LUMPUR: IHH Healthcare Bhd's net profit ...,"Feb 26, 2021 @ 8:22pm",https://www.nst.com.my/business/2021/02/669371...,"[kuala, lumpur, ihh, healthcare, bhds, net, pr..."
54,BUSINESS,Ageson's Q2 net profit increases 39pct to RM12...,KUALA LUMPUR: Ageson Bhd posted a net profit o...,"Feb 24, 2021 @ 6:31pm",https://www.nst.com.my/business/2021/02/668668...,"[kuala, lumpur, ageson, bhd, posted, net, prof..."
86,SUNDAY VIBES,MONEY THOUGHTS: Use the buy-sell filter,IS there a simple way to figure out if we're i...,"Dec 27, 2020 @ 9:15am",https://www.nst.com.my/lifestyle/sunday-vibes/...,"[simple, way, figure, were, investing, specula..."
91,PROPERTY,YTL Hospitality REIT performs better,sharen@nst.com.my\nYTL Hospitality REIT's hote...,"Nov 26, 2020 @ 7:36pm",https://www.nst.com.my/property/2020/11/644658...,"[sharennstcommy, ytl, hospitality, reit, hotel..."
